# Import libraries

In [2]:
import wandb
import pandas as pd
#from pandas_profiling import ProfileReport
from ydata_profiling import ProfileReport

# Start wandb run and load data in notebook

In [3]:
run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)
local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)

/home/carlotta/miniconda3/envs/mlflow-04641af6b7d252539cb79e908ef1f1c5164d2758/lib/python3.10/site-packages/debugpy/_vendored/force_pydevd.py:18: UserWarning: incompatible copy of pydevd already imported:
 /home/carlotta/miniconda3/envs/mlflow-04641af6b7d252539cb79e908ef1f1c5164d2758/lib/python3.10/site-packages/pydevd_plugins/__init__.py
  /home/carlotta/miniconda3/envs/mlflow-04641af6b7d252539cb79e908ef1f1c5164d2758/lib/python3.10/site-packages/pydevd_plugins/extensions/__init__.py
  /home/carlotta/miniconda3/envs/mlflow-04641af6b7d252539cb79e908ef1f1c5164d2758/lib/python3.10/site-packages/pydevd_plugins/extensions/pydevd_plugin_omegaconf.py
  warnings.warn(msg + ':\n {}'.format('\n  '.join(_unvendored)))
wandb: Currently logged in as: charly-ko. Use `wandb login --relogin` to force relogin


# Create profile report

In [6]:
profile = ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/carlotta/miniconda3/envs/mlflow-04641af6b7d252539cb79e908ef1f1c5164d2758/lib/python3.10/site-packages/ydata_profiling/model/typeset.py:125: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  not pdt.is_categorical_dtype(series)
/home/carlotta/miniconda3/envs/mlflow-04641af6b7d252539cb79e908ef1f1c5164d2758/lib/python3.10/site-packages/ydata_profiling/model/typeset.py:125: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  not pdt.is_categorical_dtype(series)
/home/carlotta/miniconda3/envs/mlflow-04641af6b7d252539cb79e908ef1f1c5164d2758/lib/python3.10/site-packages/ydata_profiling/model/typeset.py:125: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  not pdt.is_categorical_dtype(series)
/home/carlotta/miniconda3/en

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
df.head(10)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,9138664,Private Lg Room 15 min to Manhattan,47594947,Iris,Queens,Sunnyside,40.74271,-73.92493,Private room,74,2,6,2019-05-26,0.13,1,5
1,31444015,TIME SQUARE CHARMING ONE BED IN HELL'S KITCHEN...,8523790,Johlex,Manhattan,Hell's Kitchen,40.76682,-73.98878,Entire home/apt,170,3,0,NaN,NaN,1,188
2,8741020,Voted #1 Location Quintessential 1BR W Village...,45854238,John,Manhattan,West Village,40.73631,-74.00611,Entire home/apt,245,3,51,2018-09-19,1.12,1,0
3,34602077,Spacious 1 bedroom apartment 15min from Manhattan,261055465,Regan,Queens,Astoria,40.76424,-73.92351,Entire home/apt,125,3,1,2019-05-24,0.65,1,13
4,23203149,Big beautiful bedroom in huge Bushwick apartment,143460,Megan,Brooklyn,Bushwick,40.69839,-73.92044,Private room,65,2,8,2019-06-23,0.52,2,8
5,4402805,LRG 2br BKLYN APT CLOSE TO TRAINS AND PARK,22807362,Jenny,Brooklyn,Prospect-Lefferts Gardens,40.66025,-73.96270,Entire home/apt,120,3,3,2018-08-28,0.05,1,16
6,30070126,✩Prime Renovated 1/1 Apartment in Upper East S...,4968673,Sean,Manhattan,Upper East Side,40.76831,-73.95929,Entire home/apt,200,5,2,2019-05-26,0.68,1,71
7,34231172,Fully renovated brick house floor in Brooklyn,59642348,Kevin,Brooklyn,Sunset Park,40.64550,-74.01262,Entire home/apt,95,1,9,2019-07-08,9.00,1,106
8,5856760,"Renovated 1BR in exciting, convenient area",29408349,Chad,Manhattan,Chinatown,40.71490,-73.99976,Entire home/apt,179,5,7,2017-04-18,0.14,1,0
9,7929441,Beautiful Loft w/ Waterfront View!,1453898,Anthony,Brooklyn,Williamsburg,40.71268,-73.96676,Private room,105,2,232,2019-06-19,5.00,3,64


# Peculiarities
* in columns 'last_review' and 'reviews_per_month' has missing values (20.6%). This needs to be handeled in interence pipeline. 
* column 'last_review' is a date but has a string format
* column 'prise' has zeros and some very high prices. Agreed with stakeholders to consider minimum price of 10 dollar and maximum price of 350 dollar

# Fix problems in data 
## Drop outliers in 'price' column

In [8]:
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()

## Convert last_review to datetime

In [9]:
df['last_review']=pd.to_datetime(df['last_review'])

# Check pandas profil after correction

In [10]:
profile2 = ProfileReport(df)
profile2.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/carlotta/miniconda3/envs/mlflow-04641af6b7d252539cb79e908ef1f1c5164d2758/lib/python3.10/site-packages/ydata_profiling/model/typeset.py:125: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  not pdt.is_categorical_dtype(series)
/home/carlotta/miniconda3/envs/mlflow-04641af6b7d252539cb79e908ef1f1c5164d2758/lib/python3.10/site-packages/ydata_profiling/model/typeset.py:125: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  not pdt.is_categorical_dtype(series)
/home/carlotta/miniconda3/envs/mlflow-04641af6b7d252539cb79e908ef1f1c5164d2758/lib/python3.10/site-packages/ydata_profiling/model/typeset.py:125: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  not pdt.is_categorical_dtype(series)
/home/carlotta/miniconda3/en

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19001 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              19001 non-null  int64         
 1   name                            18994 non-null  object        
 2   host_id                         19001 non-null  int64         
 3   host_name                       18993 non-null  object        
 4   neighbourhood_group             19001 non-null  object        
 5   neighbourhood                   19001 non-null  object        
 6   latitude                        19001 non-null  float64       
 7   longitude                       19001 non-null  float64       
 8   room_type                       19001 non-null  object        
 9   price                           19001 non-null  int64         
 10  minimum_nights                  19001 non-null  int64         
 11  number_

# End wandb run

In [ ]:
run.finish()